# Image preprocessing

In [15]:
import os
import string
import matplotlib.pyplot as plt
import cv2
from PIL import Image

%matplotlib inline
#plt.style.use('fivethirtyeight')
#plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 14

In [2]:
ROOT = "../input/"

In [9]:
# Random image name generator
def get_rand_name(size=30, chars=string.ascii_letters + string.digits):
    return ''.join(random.choice(chars) for x in range(size))

In [10]:
# Image cropping  
def apply_crop(height, width, step_h, step_w,
               frame_in_path=None, 
               mask_in_path=None,
               frame_out_path=None,
               mask_out_path=None,
               center_crop=False):
    
    if not (os.path.isdir(frame_in_path) and
            os.path.isdir(mask_in_path) and
            os.path.isdir(frame_out_path) and
            os.path.isdir(mask_out_path)):
        raise Exception(f"Failed to load one of the given path")
    if not (height and width):
        raise Exception(f"Dimension not matched ({height}, {width})")
    if not (step_h and step_w):
        raise Exception(f"Step {step_h} or {step_w} not supported")
        
    for filename in os.listdir(frame_in_path):
        # Load frames and masks from npy source 
        frame_in = cv2.imread(os.path.join(frame_in_path, filename))
        mask_in = np.load(os.path.join(mask_in_path, filename.split('.')[0] + '.npy')) 

        for w in range(0, width, step_w):
            for h in range (0, height, step_h):
                frame_out = frame_in[h:h+step_h, w:w+step_w]
                mask_out = mask_in[h:h+step_h, w:w+step_w]
                rand_name = get_rand_name()
                cv2.imwrite(os.path.join(frame_out_path, rand_name + '.png'), frame_out)
                cv2.imwrite(os.path.join(mask_out_path, rand_name + '.png'), mask_out)

In [ ]:
# Crop images & mask(npy files)
'''
apply_crop(720, 1280, 360, 320, 
          frame_in_path=ROOT + 'source/generated-v2-01/JPEGImages/',
          mask_in_path=ROOT + 'source/generated-v2-01/SegmentationClass/', 
          frame_out_path=ROOT + 'source/sliced-v2-01/images/',
          mask_out_path=ROOT + 'source/sliced-v2-01/mask/')
'''

## Image augmentation using Ablumentations

In [16]:
# Augmentation using albumentation 
# https://albumentations.readthedocs.io/en/latest/examples.html
import albumentations as alb
import random 
import numpy as np

In [21]:
def apply_augmentation(p=0.7, frame_in_path=None, mask_in_path=None, frame_out_path=None, mask_out_path=None):
    # define augmentation pipeline
    pipeline = alb.Compose([
        alb.VerticalFlip(),
        alb.HorizontalFlip(),
        alb.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=1),
        alb.ElasticTransform(border_mode=cv2.BORDER_REFLECT_101, alpha_affine=40, p=1),
        alb.OneOf([
            alb.GaussianBlur(p=0.7, blur_limit=3),
            alb.RandomRain(p=0.7, brightness_coefficient=0.6, drop_width=1, blur_value=5),
            alb.RandomSnow(p=0.7, brightness_coeff=1, snow_point_lower=0.3, snow_point_upper=0.5),
            alb.RandomShadow(p=0.6, num_shadows_lower=1, num_shadows_upper=1, 
                            shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1)),
            alb.RandomFog(p=0.7, fog_coef_lower=0.7, fog_coef_upper=0.8, alpha_coef=0.1)
        ], p=0.8),
        alb.OneOf([
            alb.CLAHE(clip_limit=2),
            alb.IAASharpen(),
            alb.IAAEmboss(),
            alb.RandomBrightnessContrast(),
        ], p=0.6),
    ], p=p)
    
    # Apply pipeline for randomly picked image for 800 trial -> to generate 800 images
    for _ in range(800):
        # Shuffle out image list
        frame_list = os.listdir(frame_in_path)
        random.shuffle(frame_list)
        index = random.randint(0, len(frame_list)-1)
        # Pick one image 
        frame_id = frame_list[index]
        # Apply augmentation to the coosen image
        frame_in = cv2.imread(frame_in_path + frame_id)
        mask_in = cv2.imread(mask_in_path + frame_id)
        # Fit pipeline 
        augmented = pipeline(image = frame_in, mask = mask_in)
        # Get outcomes 
        frame_out, mask_out = augmented["image"], augmented["mask"]
        # Gen. out filename 
        f_name = get_rand_name()
        # Write file to out dir 
        cv2.imwrite(frame_out_path + f_name + '.png', frame_out)
        cv2.imwrite(mask_out_path + f_name + '.png', mask_out)

In [26]:
# Same as above, used for experimenting unique trasnformation
def apply_augmentation_exp(p=0.9, frame_in_path=None, mask_in_path=None, frame_out_path=None, mask_out_path=None):
    # define augmentation pipeline
    pipeline = alb.Compose([
       alb.RandomRain(p=0.7, brightness_coefficient=0.6, drop_width=1, blur_value=5),
    ], p=p)
    
    # Apply pipeline for x randomly picked images
    for _ in range(20):
        # Shuffle out image list
        frame_list = os.listdir(frame_in_path)
        random.shuffle(frame_list)
        index = random.randint(0, len(frame_list)-1)
        # Pick one image 
        frame_id = frame_list[index]
        # Apply augmentation to the coosen image
        frame_in = cv2.imread(frame_in_path + frame_id)
        mask_in = cv2.imread(mask_in_path + frame_id)
        # Fit pipeline 
        augmented = pipeline(image = frame_in, mask = mask_in)
        # Get outcomes 
        frame_out, mask_out = augmented["image"], augmented["mask"]
        # Gen. out filename 
        f_name = get_rand_name()
        # Write file to out dir 
        cv2.imwrite(frame_out_path + f_name + '.png', frame_out)
        cv2.imwrite(mask_out_path + f_name + '.png', mask_out)
        
apply_augmentation_exp(p=0.9, 
                       frame_in_path=ROOT + 'source/sliced-all/images/', 
                       mask_in_path=ROOT + 'source/sliced-all/mask/', 
                       frame_out_path=ROOT + 'source/augmented-all/images/', 
                       mask_out_path=ROOT + 'source/augmented-all/mask/')